In [2]:
import numpy as np

In [5]:
# Datos de pruebas
# Rene Manzano


# Coordenadas iniciales (aproximaciones iniciales)
puntos = np.array([
    ['10', [-922886.72715334, -5951569.11657866, 2099281.85726845]],
    ['11', [-923608.04171829, -5951534.04024824, 2099036.70315641]],
    ['4', [-924151.68281468, -5952372.84075829, 2096541.10163531]],
    ['3', [-924347.7783152, -5952308.43195145, 2096629.12382028]],
    ['8', [-921992.37525752, -5952223.15849636, 2097964.20772335]],
    ['9', [-921980.7985837, -5952077.42737261, 2098347.62710959]],
],dtype=object)

# Distancias medidas
observaciones = np.array([
    ['10', '11', 762.566],  # Distancia medida entre 10 y 11
    ['4', '3', 224.384],    # Distancia medida entre 4 y 3
    ['11', '4', 2688.1995], # Distancia medida entre 11 y 4
    ['8', '4', 2590.4349],  # Distancia medida entre 8 y 4
    ['8', '9', 410.188],    # Distancia medida entre 8 y 9
    ['8', '11', 2057.4457],  # Distancia medida entre 8 y 11
], dtype=object)

In [16]:
def parcial_coeficients_obs_row(XA, XB, LAB):
    """
    Calcula una fila de la matriz A para la ecuación de distancia linearizada entre dos puntos.

    Parámetros:
    XA : array_like
        Coordenadas iniciales del punto A (aproximaciones iniciales).
    XB : array_like
        Coordenadas iniciales del punto B (aproximaciones iniciales).
    LAB : float
        Distancia medida entre los puntos A y B.

    Devuelve:
    A_row : list
        Fila de la matriz A con las derivadas parciales.
    W : float
        Vector de observaciones (diferencia entre distancia medida y distancia calculada).
    """

    if len(XA) == 2 or len(XB) == 2:
        # Diferencias de coordenadas en 2D
        dX = XB[0] - XA[0]
        dY = XB[1] - XA[1]

        # Distancia calculada entre los puntos A y B en 2D
        AB0 = np.sqrt(dX**2 + dY**2)

        # Derivadas parciales en 2D
        dX_A = -dX / AB0
        dY_A = -dY / AB0
        dX_B = dX / AB0
        dY_B = dY / AB0

        # Fila de la matriz A en 2D
        partial_pt1 = [dX_A, dY_A]
        partial_pt2 = [dX_B, dY_B]

    else:
        # Diferencias de coordenadas en 3D
        dX = XB[0] - XA[0]
        dY = XB[1] - XA[1]
        dZ = XB[2] - XA[2]

        # Distancia calculada entre los puntos A y B en 3D
        AB0 = np.sqrt(dX**2 + dY**2 + dZ**2)

        # Derivadas parciales en 3D
        dX_A = -dX / AB0
        dY_A = -dY / AB0
        dZ_A = -dZ / AB0
        dX_B = dX / AB0
        dY_B = dY / AB0
        dZ_B = dZ / AB0

        # Fila de la matriz A en 3D
        partial_pt1 = [dX_A, dY_A, dZ_A]
        partial_pt2 = [dX_B, dY_B, dZ_B]

    # Vector de observaciones W (diferencia entre distancia medida y distancia calculada)
    W = AB0 - LAB

    return partial_pt1, partial_pt2, W

In [17]:
def calcular_columnas_matrix_a(puntos, distancias, condiciones = False):
    """
    Calcula el número de columnas de la matriz A basada en los puntos y las distancias.

    :param puntos: ndarray de puntos con sus coordenadas.
    :param distancias: Lista de tuplas con los puntos y la distancia entre ellos.
    :return: Número de columnas de la matriz A.
    """
    num_puntos = puntos.shape[0]
    pt = puntos[0]
    dim_punto = len(pt[1]) if num_puntos > 0 else 0

    if condiciones:
        return num_puntos * dim_punto + len(distancias)
    else:
        return num_puntos * dim_punto

def crear_matriz(puntos, distancias, condiciones = False):
    """
    Crea una matriz inicializada con ceros basada en los puntos y distancias.

    :param puntos: ndarray de puntos con sus coordenadas.
    :param distancias: Lista de tuplas con los puntos y la distancia entre ellos.
    :return: Matriz inicializada con ceros.
    """
    columnas = calcular_columnas_matrix_a(puntos, distancias, condiciones)
    filas = len(distancias)
    return np.zeros((filas, columnas))

def obtener_indices_puntos(puntos):
    """
    Obtiene los índices correspondientes a cada punto en la matriz A.

    :param puntos: ndarray de puntos con sus coordenadas.
    :return: Diccionario con los índices de los puntos.
    """
    indices_puntos = {}
    dim_punto = len(puntos[0, 1])
    for i, punto in enumerate(puntos[:, 0]):
        indices_puntos[punto] = range(i * dim_punto, (i + 1) * dim_punto)
    return indices_puntos

def llenar_matriz(puntos, distancias, condiciones = False):
    """
    Llena la matriz A con los coeficientes parciales y las observaciones.

    :param puntos: ndarray de puntos con sus coordenadas.
    :param distancias: Lista de tuplas con los puntos y la distancia entre ellos.
    :return: Matriz A llena y lista W de coeficientes parciales.
    """
    # Crear la matriz inicializada
    matriz = crear_matriz(puntos, distancias, condiciones)
    
    
    # Obtener los índices de los puntos
    indices_puntos = obtener_indices_puntos(puntos)
    
    # Crear un diccionario para acceder rápidamente a las coordenadas por nombre
    coord_dict = {punto: np.array(coords) for punto, coords in puntos}

    W = []
    for i, (p1, p2, dist) in enumerate(distancias):
        try:
            coord_p1 = coord_dict[p1]
            coord_p2 = coord_dict[p2]
        except KeyError as e:
            raise KeyError(f"Punto {e} no encontrado en 'puntos'. Verifica que todos los identificadores en 'distancias' están presentes en 'puntos'.")
        
        partial1, partial2, w = parcial_coeficients_obs_row(coord_p1, coord_p2, dist)


        # Llenar la matriz con las coordenadas de los puntos
        matriz[i, indices_puntos[p1]] = partial1  # Coordenadas del primer punto
        matriz[i, indices_puntos[p2]] = partial2  # Coordenadas del segundo punto

        if condiciones:
            # Añadir -1 en la posición correspondiente a la distancia
            matriz[i, -len(distancias) + i] = -1

        # Agregar w a la lista W
        W.append(w)

    return matriz, W


In [18]:
def weight(sigma, puntos, distancias, condiciones = False):
    if condiciones:
        length = m.calcular_columnas_matrix_a(puntos, distancias, condiciones)
    else: 
        length = len(distancias)
    return np.diag(np.full(length, 1/(sigma)**2))

In [19]:
condiciones = True

A, f = llenar_matriz(puntos, observaciones, condiciones)
# W = wg.weight(0.003, d.puntos, d.observaciones, condiciones)
W = np.diag([.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.000004,.000004,.000004,.000004,.000004,.000004])

# print(f"Matrix de diseño {A.shape}: \n{A}")
# print(f"\nMatrix de Pesos {W.shape}: \n{W}")
# print(f"\nVector de residuos (1, {len(f)}): \n{f}")

M = np.linalg.inv(np.dot(A,np.dot(W,A.T)))
# print(f"M: \n{M}")

K = np.dot(M,f)
# print(f"K: \n{K}")

V = np.dot(np.dot(W,A.T),K)
print(f"V: \n{V}")

V: 
[-5.20537373e-02  2.53128687e-03 -1.76915709e-02 -2.07018633e-01
  1.15983644e-01  2.11048982e-01  5.33993100e-02  6.13708288e-03
  4.26807144e-02  2.11987274e-02 -6.96285604e-03 -9.51555901e-03
  1.85773757e-01 -1.01331565e-01 -1.83485651e-01 -1.29942396e-03
 -1.63575925e-02 -4.30369157e-02  4.40290134e-04  1.94057851e-04
 -1.60991798e-04  7.54988406e-04  3.68472345e-04 -2.68151995e-03]


In [20]:
def parcial_coeficients_obs_row(XA, XB, LAB):
    """
    Calcula una fila de la matriz A para la ecuación de distancia linearizada entre dos puntos.

    Parámetros:
    XA : array_like
        Coordenadas iniciales del punto A (aproximaciones iniciales).
    XB : array_like
        Coordenadas iniciales del punto B (aproximaciones iniciales).
    LAB : float
        Distancia medida entre los puntos A y B.

    Devuelve:
    A_row : list
        Fila de la matriz A con las derivadas parciales.
    W : float
        Vector de observaciones (diferencia entre distancia medida y distancia calculada).
    """

    if len(XA) == 2 or len(XB) == 2:
        # Diferencias de coordenadas en 2D
        dX = XB[0] - XA[0]
        dY = XB[1] - XA[1]

        # Distancia calculada entre los puntos A y B en 2D
        AB0 = np.sqrt(dX**2 + dY**2)

        # Derivadas parciales en 2D
        dX_A = -dX / AB0
        dY_A = -dY / AB0
        dX_B = dX / AB0
        dY_B = dY / AB0

        # Fila de la matriz A en 2D
        partial_pt1 = [dX_A, dY_A]
        partial_pt2 = [dX_B, dY_B]

    else:
        # Diferencias de coordenadas en 3D
        dX = XB[0] - XA[0]
        dY = XB[1] - XA[1]
        dZ = XB[2] - XA[2]

        # Distancia calculada entre los puntos A y B en 3D
        AB0 = np.sqrt(dX**2 + dY**2 + dZ**2)

        # Derivadas parciales en 3D
        dX_A = -dX / AB0
        dY_A = -dY / AB0
        dZ_A = -dZ / AB0
        dX_B = dX / AB0
        dY_B = dY / AB0
        dZ_B = dZ / AB0

        # Fila de la matriz A en 3D
        partial_pt1 = [dX_A, dY_A, dZ_A]
        partial_pt2 = [dX_B, dY_B, dZ_B]

    # Vector de observaciones W (diferencia entre distancia medida y distancia calculada)
    W = AB0 - LAB

    return partial_pt1, partial_pt2, W

def calcular_columnas_matrix_a(puntos, distancias, condiciones = False):
    """
    Calcula el número de columnas de la matriz A basada en los puntos y las distancias.

    :param puntos: ndarray de puntos con sus coordenadas.
    :param distancias: Lista de tuplas con los puntos y la distancia entre ellos.
    :return: Número de columnas de la matriz A.
    """
    num_puntos = puntos.shape[0]
    pt = puntos[0]
    dim_punto = len(pt[1]) if num_puntos > 0 else 0

    if condiciones:
        return num_puntos * dim_punto + len(distancias)
    else:
        return num_puntos * dim_punto

def crear_matriz(puntos, distancias, condiciones = False):
    """
    Crea una matriz inicializada con ceros basada en los puntos y distancias.

    :param puntos: ndarray de puntos con sus coordenadas.
    :param distancias: Lista de tuplas con los puntos y la distancia entre ellos.
    :return: Matriz inicializada con ceros.
    """
    columnas = calcular_columnas_matrix_a(puntos, distancias, condiciones)
    filas = len(distancias)
    return np.zeros((filas, columnas))

def obtener_indices_puntos(puntos):
    """
    Obtiene los índices correspondientes a cada punto en la matriz A.

    :param puntos: ndarray de puntos con sus coordenadas.
    :return: Diccionario con los índices de los puntos.
    """
    indices_puntos = {}
    dim_punto = len(puntos[0, 1])
    for i, punto in enumerate(puntos[:, 0]):
        indices_puntos[punto] = range(i * dim_punto, (i + 1) * dim_punto)
    return indices_puntos

def llenar_matriz(puntos, distancias, condiciones = False):
    """
    Llena la matriz A con los coeficientes parciales y las observaciones.

    :param puntos: ndarray de puntos con sus coordenadas.
    :param distancias: Lista de tuplas con los puntos y la distancia entre ellos.
    :return: Matriz A llena y lista W de coeficientes parciales.
    """
    # Crear la matriz inicializada
    matriz = crear_matriz(puntos, distancias, condiciones)
    
    
    # Obtener los índices de los puntos
    indices_puntos = obtener_indices_puntos(puntos)
    
    # Crear un diccionario para acceder rápidamente a las coordenadas por nombre
    coord_dict = {punto: np.array(coords) for punto, coords in puntos}

    W = []
    for i, (p1, p2, dist) in enumerate(distancias):
        try:
            coord_p1 = coord_dict[p1]
            coord_p2 = coord_dict[p2]
        except KeyError as e:
            raise KeyError(f"Punto {e} no encontrado en 'puntos'. Verifica que todos los identificadores en 'distancias' están presentes en 'puntos'.")
        
        partial1, partial2, w = parcial_coeficients_obs_row(coord_p1, coord_p2, dist)


        # Llenar la matriz con las coordenadas de los puntos
        matriz[i, indices_puntos[p1]] = partial1  # Coordenadas del primer punto
        matriz[i, indices_puntos[p2]] = partial2  # Coordenadas del segundo punto

        if condiciones:
            # Añadir -1 en la posición correspondiente a la distancia
            matriz[i, -len(distancias) + i] = -1

        # Agregar w a la lista W
        W.append(w)

    return matriz, W

def weight(sigma, puntos, distancias, condiciones = False):
    if condiciones:
        length = m.calcular_columnas_matrix_a(puntos, distancias, condiciones)
    else: 
        length = len(distancias)
    return np.diag(np.full(length, 1/(sigma)**2))

# Coordenadas iniciales (aproximaciones iniciales)
puntos = np.array([
    ['10', [-922886.72715334, -5951569.11657866, 2099281.85726845]],
    ['11', [-923608.04171829, -5951534.04024824, 2099036.70315641]],
    ['4', [-924151.68281468, -5952372.84075829, 2096541.10163531]],
    ['3', [-924347.7783152, -5952308.43195145, 2096629.12382028]],
    ['8', [-921992.37525752, -5952223.15849636, 2097964.20772335]],
    ['9', [-921980.7985837, -5952077.42737261, 2098347.62710959]],
],dtype=object)

# Distancias medidas
observaciones = np.array([
    ['10', '11', 762.566],  # Distancia medida entre 10 y 11
    ['4', '3', 224.384],    # Distancia medida entre 4 y 3
    ['11', '4', 2688.1995], # Distancia medida entre 11 y 4
    ['8', '4', 2590.4349],  # Distancia medida entre 8 y 4
    ['8', '9', 410.188],    # Distancia medida entre 8 y 9
    ['8', '11', 2057.4457],  # Distancia medida entre 8 y 11
], dtype=object)

condiciones = True

A, f = llenar_matriz(puntos, observaciones, condiciones)
# W = wg.weight(0.003, d.puntos, d.observaciones, condiciones)
W = np.diag([.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.0005,.000004,.000004,.000004,.000004,.000004,.000004])

# print(f"Matrix de diseño {A.shape}: \n{A}")
# print(f"\nMatrix de Pesos {W.shape}: \n{W}")
# print(f"\nVector de residuos (1, {len(f)}): \n{f}")

M = np.linalg.inv(np.dot(A,np.dot(W,A.T)))
# print(f"M: \n{M}")

K = np.dot(M,f)
# print(f"K: \n{K}")

V = np.dot(np.dot(W,A.T),K)
print(f"V: \n{V}")


V: 
[-5.20537373e-02  2.53128687e-03 -1.76915709e-02 -2.07018633e-01
  1.15983644e-01  2.11048982e-01  5.33993100e-02  6.13708288e-03
  4.26807144e-02  2.11987274e-02 -6.96285604e-03 -9.51555901e-03
  1.85773757e-01 -1.01331565e-01 -1.83485651e-01 -1.29942396e-03
 -1.63575925e-02 -4.30369157e-02  4.40290134e-04  1.94057851e-04
 -1.60991798e-04  7.54988406e-04  3.68472345e-04 -2.68151995e-03]
